<center><img src="../images/DLI Header.png" alt="Header" style="width: 400px;"/></center>

# Jetson Nano で AI をはじめる
### インタラクティブ型回帰ツール

このノートブックは、NVIDIA Deep Learning Institute (DLI) コース「Jetson Nano で AI をはじめる」の一部として提供されるインタラクティブなデータ収集、学習、およびテスト ツールです。 オンライン DLI コース ページで提供される詳細な手順と併せて、 Jetson Nano で実行するように設計されています。 

ツールを開始するには、次の **カメラ** および **タスク** のセルの設定を行ってから、残りの全てのセルを実行します。ノートブックの下部にあるインタラクティブ ツール ウィジェットが表示されます。 このツールを使用して、データを収集し、データを追 加し、データを学習し、繰り返しインタラクティブな方法でデータをテストできます！ 

このノートブックの説明は学習を能率的に進めるため、意図的に最小限に抑えられています。ツールの操作とプロジェクト作成の詳細については、DLI コース ページをご覧ください。

### カメラ
はじめに、カメラを使用するため、あなたの Jetson Nano に接続しているカメラの種類 (USB まはた CSI) に応じてコメント行を編集します。このセルの実行には数十秒かかる場合があります。
<div style="border:2px solid black; background-color:#e3ffb3; font-size:12px; padding:8px; margin-top: auto;"><i>
    <h4><i>ヒント</i></h4>
<p>CSI カメラ、または USB カメラのインスタンスは、一度に 1 つしか存在できません。新しいプロジェクトを開始して新しいカメラ インスタンス を作成する前に、まず既存のカメラ インスタンスを解放する必要があります。これを行うには、JupyterLab プルダウン メニューからノートブックのカーネルをシャットダウンします： <strong>Kernel->Shutdown Kernel</strong> 、それから <strong>Kernel->Restart Kernel</strong> で再起動します。</p>
<ul><code>sudo systemctl restart nvargus-daemon</code> コマンドで、パスワード :<code>dlinano</code> を使用するとカメラ デーモンを強制的にリセットできます。</ul>

In [1]:
# Full reset of the camera
!echo 'dlinano' | sudo -S systemctl restart nvargus-daemon && printf '\n'
# Check device number
!ls -ltrh /dev/video*

# USB Camera (Logitech C270 webcam)
from jetcam.usb_camera import USBCamera
camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

# CSI Camera (Raspberry Pi Camera Module V2)
# from jetcam.csi_camera import CSICamera
# camera = CSICamera(width=224, height=224)

camera.running = True
print("camera created")

[sudo] taityo1999 のパスワード: すみません、もう一度試してください。
[sudo] taityo1999 のパスワード: 
sudo: 1 回パスワード試行を間違えました
ls: '/dev/video*' にアクセスできません: そのようなファイルやディレクトリはありません


ModuleNotFoundError: No module named 'jetcam'

### タスク
次に、プロジェクト `TASK (タスク)` と収集するデータの `CATEGORIES (カテゴリー)` を定義します。必要に応じてお好みの `DATASETS (データセット)` をスペースで区切って複数定義できます。 

ビルドする XY 回帰タスク (the XY regression task) に関連する行のコメントを削除または編集しセルを実行します。このセルの実行は数秒で完了します。

In [ ]:
import torchvision.transforms as transforms
from dataset import XYDataset

TASK = 'face'
# TASK = 'diy'

CATEGORIES = ['nose', 'left_eye', 'right_eye']
# CATEGORIES = [ 'diy_1', 'diy_2', 'diy_3']

DATASETS = ['A', 'B']
# DATASETS = ['A', 'B', 'C']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS)
    
print("{} task with {} categories defined".format(TASK, CATEGORIES))

### データ収集
次のセルを実行して、データ収集ツール ウィジェットを作成します。このセルの実行は数秒で完了します。

In [8]:
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget


# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
with open("../images/ready_img.jpg", "rb") as file:
    default_image = file.read()
snapshot_widget = ipywidgets.Image(value=default_image, width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')


def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save to disk
        dataset.save_entry(category_widget.value, camera.value, x, y)
        
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = dataset.get_count(category_widget.value)
        
camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    dataset_widget,
    category_widget,
    count_widget
])

# display(data_collection_widget)
print("data_collection_widget created")

ModuleNotFoundError: No module named 'jetcam'

### モデル
次のセルを実行してニューラルネットワークを定義し、プロジェクトに必要な出力に一致するように全結合層 (`fc`) を調整します。このセルの実行には数十秒かかる場合があります。

In [ ]:
import torch
import torchvision

device = torch.device('cuda')
output_dim = 2 * len(dataset.categories)  # x, y coordinate for each category

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

model = model.to(device)

model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value='my_xy_model.pth')

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

model_save_button.click()

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])

# display(model_widget)
print("model configured and model_widget created")

### ライブ実行
次のセルを実行して、ライブ実行ウィジェットを設定します。このセルの実行は数秒で完了します。

In [10]:
import threading
import time
import cv2
import ipywidgets

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
with open("../images/ready_img.jpg", "rb") as file:
    default_image = file.read()
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height, value=default_image)

def live(state_widget, model, camera, prediction_widget):
    global dataset
    while state_widget.value == 'live':
        image = camera.value
        preprocessed = preprocess(image)
        output = model(preprocessed).detach().cpu().numpy().flatten()
        category_index = dataset.categories.index(category_widget.value)
        x = output[2 * category_index]
        y = output[2 * category_index + 1]
        
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))
        
        prediction = image.copy()
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
            
def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

# display(live_execution_widget)
print("live_execution_widget created")

NameError: name 'camera' is not defined

### 学習と評価
次のセルを実行してトレーナーと、それを制御するウィジェットを定義します。このセルの実行には数十秒かかる場合があります。

In [3]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()

        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, category_idx, xy in iter(train_loader):
                # send data to device
                images = images.to(device)
                xy = xy.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute MSE loss over x, y coordinates for associated categories
                loss = 0.0
                for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                    loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
                loss /= len(category_idx)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                count = len(category_idx.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    state_widget.value = 'live'
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

# display(train_eval_widget)
print("trainer configured and train_eval_widget created")

NameError: name 'torch' is not defined

### インタラクティブ型ツールの表示

インタラクティブ型ツールには、データ収集、学習、およびテスト用のウィジェットが含まれています。

<center><img src="../images/regression_XY_tool_key2.png" alt="tool key" width=500/></center>
<br>
<center><img src="../images/regression_XY_tool_key1.png" alt="tool key"/></center>

次のセルを実行して、完全なインタラクティブ ウィジェットを作成、および表示します。オンライン DLI コース ページの指示に従ってプロジェクトをビルドします。

In [4]:
# Combine all the widgets into one display
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget, live_execution_widget]), 
    train_eval_widget,
    model_widget
])

display(all_widget)

NameError: name 'ipywidgets' is not defined

<center><img src="../images/DLI Header.png" alt="Header" style="width: 400px;"/></center>